# Partisan Voter Index (PVI) calculations for Massachusetts Legislative Districts

The Partisan Voter Index or PVI was created by the Cook Political Report to create a common metric for comparing the partisanship or Democratic/Republican lean of legislative districts. The Cook Political Report primarily uses the PVI for comparing U.S. House districts, but it can also be used to compare state legislative districts.

PVI is calculated by taking the Democratic percentage of the over the last two presidential elections for the area you are considering and then subtracting the average for the U.S. as a whole.

In [1]:
US_VOTES = {
    "Obama_12": 65915796,
    "Romney_12": 60933500,
    "Clinton_16": 65853516,
    "Trump_16": 62984824,
    }

def dem_pct(d, r):
    """The Democratic percentage of the vote, given Dem and GOP vote totals."""
    return (d / (d + r))

US_DEM_PCT = dem_pct(US_VOTES["Obama_12"] + US_VOTES["Clinton_16"],
                     US_VOTES["Romney_12"] + US_VOTES["Trump_16"])

def calc_pvi(df):
    """Numeric PVI given vote totals from last two presidential elections."""
    dem = df["Obama_12"] + df["Clinton_16"]
    gop = df["Romney_12"] + df["Trump_16"]
    pvi = (dem_pct(dem, gop) - US_DEM_PCT) * 100
    return pvi

def pvi_string(pvi):
    """String representative of numeric PVI value."""
    if pvi <= -0.5:
        s = "R+{:.0f}".format(abs(pvi))
    elif pvi >= 0.5:
        s = "D+{:.0f}".format(abs(pvi))
    else:
        s = "EVEN"
    return s

For example, we can calculate the PVI for Massachusetts as a whole using the vote totals from the 2012 and 2016 presidential elections.

In [2]:
MA_VOTES = {
    "Obama_12": 1921290,
    "Romney_12": 1188314,
    "Clinton_16": 1995196,
    "Trump_16": 1090893,
    }

MA_PVI_N = calc_pvi(MA_VOTES)
MA_PVI_N

11.677772318663537

So Massachusetts is about 11.7 points more Democratic than the country as a whole using the PVI metric.

There is a formatting convention for PVI to round the numeric value to an integer and show negative/GOP-leaning PVI with an "R+" and positive/Dem-leaning PVI with a "D+".

In [3]:
MA_PVI = pvi_string(MA_PVI_N)
MA_PVI

'D+12'

## Precinct-level presidential results

In order to calculate PVI across various legislative districts in Massachusetts, we will start by downloading the precinct-level presidential election data for 2012 and 2016 from the awesome [electionstats.state.ma.us](http://electionstats.state.ma.us) website created by Adam Friedman and supported by the Massachusetts Secretary of the Commonwealth.

Once we have the precinct-level data, we can map the precincts to legislative districts and then do the PVI calculations.

In [4]:
import pandas as pd
pd.set_option("display.max_rows", 500)

def read_pd43(url, set_index=True):
    """Read a precinct-level CSV file from PD43 site."""
    # Force Ward and Pct to string for consistency.
    p = pd.read_csv(url, dtype={"Ward": str, "Pct": str}, thousands=",", skiprows=[1])
    # Remove TOTALS row
    p = p[p["City/Town"] != "TOTALS"]
    # Optionally set the index to three precinct-identifying columns
    if set_index:
        p = p.set_index(["City/Town", "Ward", "Pct"])
    return p

p12 = read_pd43("http://electionstats.state.ma.us/elections/download/22515/precincts_include:1/")
p12.head(20)

Obama/ Biden  Romney/ Ryan  Johnson/ Gray  Stein/ Honkala  \
City/Town Ward Pct                                                              
Abington  -    1             844           705              6               7   
               2             742           741             18               8   
               3             815           801             13              11   
               4             899           904              8               9   
               5             844           910              8               2   
Acton     -    1            1228           583             18               5   
               2            1216           722             19              14   
               3            1378           699             34              24   
               4            1462           636             25              20   
               5            1454           565             25              11   
               6            1134           670             15               9   
Acushnet  -    1            1034           574             17               8   
               2             959           803             25              11   
               3            1053           819             17               7   
Adams     -    1             525           164             17               1   
               2             607           172              9               6   
               3             582           169             10               3   
               4             590           129              9               6   
               5             709           185              8               9   
Agawam    -    1             850           665             13               7   

                    All Others  No Preference  Blanks  Total Votes Cast  
City/Town Ward Pct                                                       
Abington  -    1             4              0       5              1571  
               2             4              0       7              1520  
               3             2              0       6              1648  
               4             5              0       5              1830  
               5             5              0       5              1774  
Acton     -    1             3              0       8              1845  
               2             3              0       4              1978  
               3             0              0       8              2143  
               4             0              0       6              2149  
               5             8              0       8              2071  
               6             3              0       5              1836  
Acushnet  -    1            13              0      23              1669  
               2             4              0      19              1821  
               3             1              0      11              1908  
Adams     -    1             0              0       1               708  
               2             2              0       5               801  
               3             2              0       2               768  
               4             3              0       3               740  
               5             2              0       1               914  
Agawam    -    1             6              0       5              1546

We are only interested in the Democratic and Republican vote totals.

In [5]:
p12 = p12.rename(columns={"Obama/ Biden": "Obama_12", "Romney/ Ryan": "Romney_12"})
p12 = p12[["Obama_12", "Romney_12"]]
p12.head(20)

Obama_12  Romney_12
City/Town Ward Pct                     
Abington  -    1         844        705
               2         742        741
               3         815        801
               4         899        904
               5         844        910
Acton     -    1        1228        583
               2        1216        722
               3        1378        699
               4        1462        636
               5        1454        565
               6        1134        670
Acushnet  -    1        1034        574
               2         959        803
               3        1053        819
Adams     -    1         525        164
               2         607        172
               3         582        169
               4         590        129
               5         709        185
Agawam    -    1         850        665

Repeat the same process for 2016 presidential result data.

In [6]:
p16 = read_pd43("http://electionstats.state.ma.us/elections/download/40060/precincts_include:1/")
p16 = p16.rename(columns={"Clinton/ Kaine": "Clinton_16", "Trump/ Pence": "Trump_16"})
p16 = p16[["Clinton_16", "Trump_16"]]
p16.head(20)

Clinton_16  Trump_16
City/Town Ward Pct                      
Abington  -    1           818       717
               2           739       785
               3           773       808
               4           877       878
               5           908       829
Acton     -    1          1459       438
               2          1550       480
               3          1580       435
               4          1675       414
               5          1580       387
               6          1354       401
Acushnet  -    1           783       786
               2           756      1043
               3           862      1005
Adams     -    1           439       255
               2           476       270
               3           475       268
               4           446       233
               5           577       259
Agawam    -    1           631       827

Now we combine the 2012 and 2016 data together.

In [7]:
p = p12.join(p16)
p.head(20)

Obama_12  Romney_12  Clinton_16  Trump_16
City/Town Ward Pct                                           
Abington  -    1         844        705         818       717
               2         742        741         739       785
               3         815        801         773       808
               4         899        904         877       878
               5         844        910         908       829
Acton     -    1        1228        583        1459       438
               2        1216        722        1550       480
               3        1378        699        1580       435
               4        1462        636        1675       414
               5        1454        565        1580       387
               6        1134        670        1354       401
Acushnet  -    1        1034        574         783       786
               2         959        803         756      1043
               3        1053        819         862      1005
Adams     -    1         525        164         439       255
               2         607        172         476       270
               3         582        169         475       268
               4         590        129         446       233
               5         709        185         577       259
Agawam    -    1         850        665         631       827

## Mapping precincts to legislative districts

In order to map the precincts to legislative districts we download results from all of the elections for that office for the year 2016 and mark each one with the district name. This requires some web page scraping using the BeautifulSoup HTML parsing library. 

In [8]:
import bs4
import requests

ELECTION_QUERY = "http://electionstats.state.ma.us/elections/search/year_from:{year}/year_to:{year}/office_id:{office_id}/stage:{stage}"
STATE_REP_ID = 8
STATE_SEN_ID = 9
US_CONG_ID = 5

def get_election_ids(year, office_id, stage):
    """Returns a list (ELECTION_ID, DISTRICT) for the given query parameters."""
    r = requests.get(ELECTION_QUERY.format(year=year, office_id=office_id, stage=stage))
    bs = bs4.BeautifulSoup(r.text, "html")
    elec_trs = bs.findAll("tr", {"class": "election_item general_party"})
    elec_ids = [district_and_id(elec_tr) for elec_tr in elec_trs]
    return elec_ids
  
def district_and_id(elec_tr):
    """Parse the ELECTION_ID and DISTRICT from the HTML element."""
    election_id = elec_tr["id"].split("-")[2]
    district = elec_tr.td.findNext("td").findNext("td").text
    return (election_id, district)

CSV_QUERY = "http://electionstats.state.ma.us/elections/download/{election_id}/precincts_include:1/"

def get_office_precincts(year, office_id, stage):
    """Query for the matching elections,
    return all of the precincts tagged with the district name.
    """
    election_list = []
    election_ids = get_election_ids(year, office_id, stage)
    for election_id, district in election_ids:
        p = read_pd43(CSV_QUERY.format(election_id=election_id), set_index=False)
        p = p[["City/Town", "Ward", "Pct"]]
        p["District"] = district
        election_list.append(p)
    all_precincts = pd.concat(election_list, ignore_index=True).drop_duplicates()
    all_precincts = all_precincts.set_index(["City/Town", "Ward", "Pct"])
    return all_precincts

In [ ]:
sr_pcts = (get_office_precincts(2016, STATE_REP_ID, "General").
           rename(columns={"District": "State_Rep"}))
ss_pcts = (get_office_precincts(2016, STATE_SEN_ID, "General")
           .rename(columns={"District": "State_Sen"}))
ush_pcts = (get_office_precincts(2016, US_CONG_ID, "General")
            .rename(columns={"District": "US_House"}))

# Combine the presidential result data with all of the district tags
pvi = p.join(ush_pcts).join(ss_pcts).join(sr_pcts).reset_index()
pvi.head(20)

## Calculate PVI for each office/district

For each office—State Representative, State Senate, and U.S. House—group by district, summing up the presidential votes and then calculate the numeric PVI and PVI string.

Begin with State Representative.

In [ ]:
sr_pvi = pvi.groupby("State_Rep").sum().reset_index()
sr_pvi["PVI_N"] = calc_pvi(sr_pvi)
sr_pvi["PVI"] = sr_pvi["PVI_N"].map(pvi_string)
sr_pvi.sort_values("PVI_N", ascending=False)

Next, State Senate.

In [ ]:
ss_pvi = pvi.groupby("State_Sen").sum().reset_index()
ss_pvi["PVI_N"] = calc_pvi(ss_pvi)
ss_pvi["PVI"] = ss_pvi["PVI_N"].map(pvi_string)
ss_pvi.sort_values("PVI_N", ascending=False)

And finally, U.S. House.

In [ ]:
ush_pvi = pvi.groupby("US_House").sum().reset_index()
ush_pvi["PVI_N"] = calc_pvi(ush_pvi)
ush_pvi["PVI"] = ush_pvi["PVI_N"].map(pvi_string)
ush_pvi.sort_values("PVI_N", ascending=False)